## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
!python -m site

sys.path = [
    '/home/e978317@WCB.AB.CA/training-learning/agents/6_mcp',
    '/home/e978317@WCB.AB.CA/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python312.zip',
    '/home/e978317@WCB.AB.CA/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12',
    '/home/e978317@WCB.AB.CA/.local/share/uv/python/cpython-3.12.11-linux-x86_64-gnu/lib/python3.12/lib-dynload',
    '/home/e978317@WCB.AB.CA/training-learning/agents/.venv/lib/python3.12/site-packages',
]
USER_BASE: '/home/e978317@WCB.AB.CA/.local' (exists)
USER_SITE: '/home/e978317@WCB.AB.CA/.local/lib/python3.12/site-packages' (doesn't exist)
ENABLE_USER_SITE: False


In [3]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

### Special note - in case you updated the OpenAI Agents framework:

OpenAI Agents SDK recently upgraded their SDK and made a breaking change (thank you, OpenAI 😂). I haven't yet updated the version in this repo to the latest version. If you've upgraded the package yourself, then you might get an error about missing positional arguments. If you get that error, you'll need to replace `await server.list_tools()` with `await server.session.list_tools()` and change the last line from `fetch_tools` to `fetch_tools.tools`. And you'll need to make this change every time you get this error! The joys of working at the bleeding edge..

# Q. what happens when you run?
```bash
uvx mcp-server-fetch
```

A. it is equal to 
```bash
python -m mcp-server-fetch
```
- The -m flag tells Python:
    “Find this module (or package) in `sys.path` and execute it as a script.”

- Python looks for the module/package name (mcp_server_fetch) inside `site-packages`.


Q .what is the difference between `sys.path` and `site-packages`?<br>
A. `site-packages` is one of pathes in `sys.path`


Q. What is `site-packages`?<br>
A. `site-packages` is the **standard directory** where Python installs **third-party packages** (i.e., packages installed with `pip`).  
When you run `pip install <package>`, Python copies the package’s code into `site-packages`.  

When you do `import <package>`, Python searches in the following order:

1. Current working directory
2. Standard library
3. **site-packages** (and any other paths listed in `sys.path`)

So `site-packages` is essentially where Python keeps all your installed add-ons.

---

## Where is `site-packages` located?

The location depends on your OS and whether you are using a **virtual environment**.

You can find it with:

```bash
python -m site
```

## `sys.path` and Its Relation to `site-packages`

#### What is `sys.path`?

- `sys.path` is a **list of directories** that Python searches when you try to `import` a module.
- It tells Python **where to look** for modules and packages.
- You can see it by running:

```python
import sys
print(sys.path)
```

example:
```python
[
    '/home/alice/myproject',
    '/usr/lib/python3.11',
    '/usr/lib/python3.11/lib-dynload',
    '/home/alice/myproject/.venv/lib/python3.11/site-packages',
]
```

#### How `sys.path` Relates to `site-packages`

- When you install a package with `pip`, it goes into **site-packages**.

- When you `import <package>`, Python searches each directory in **`sys.path`** in order.

- When it finds the package inside **site-packages**, it loads it.

- So, **site-packages** is just one of the paths in **`sys.path`** where Python looks for packages.


In [4]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get the actual H

## Q. how to start an MCP server and how to create an MCP clinet?
```python
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()
```
- `MCPServerStdio`: async context manager
- **MCPServerStdio** start your server (by running the commands in params), it creates a connection to your server over `stdio` and create a client for the server (`server`).
- `server`: client of your server. it can communicate with server through stdio stream

### what are this params?
- when you're working with MCP servers, it everything begins with these things called parameters.
-  it's actually a command. This is the command that needs to run at the command line.


#### Q. why we need `async with`? <br>
A. The key reason you cannot use a normal `with` here is because `MCPServerStdio` is an asynchronous context manager. <br>
`MCPServerStdio` is openAI agents SDK **asynchronous** context manager 

normal `with` returns type error: <br>
```bash
TypeError: 'MCPServerStdio' object does not support the context manager protocol
```

In [1]:
# # it return error: TypeError: 'MCPServerStdio' object does not support the context manager protocol
# params = {"command": "uv", "args": ["run", "accounts_server.py"]}
# with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
#     mcp_tools = server.list_tools()

In [6]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Close browser', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_resize', description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=True, destructiveHint=False, idempotentHint=None, openWorldHint=True)),
 Tool(name='browser_console_messages', description='Returns all console messages', inputSchema={'type': 'object', 'properties': {}, 'additi

### what the code below do?
```python
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools
```
this server is allowing you to equip an agent with tools so that it can read and write from your file system, but keeping it isolated to within a certain directory.


 Understanding `@modelcontextprotocol/server-filesystem` in Python Code

In your Python code:

```python
sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools
```

`@modelcontextprotocol/server-filesystem` is **not Python syntax**; it is a **Node.js package**. Here's a breakdown:

1. **Context in your code**  
   ```python
   files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}
   ```
   - `command: "npx"` → Runs the Node.js package runner (`npx`).  
   - `args: ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]` → Arguments passed to `npx`:  
     - `-y` → Automatically answer "yes" to prompts.  
     - `@modelcontextprotocol/server-filesystem` → The npm package to install/run.  
     - `sandbox_path` → Directory path where the package operates.

In [7]:

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params,client_session_timeout_seconds=60) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'type': 'number', 'description': 'If provided, returns only the last N lines of the file'}, 'head': {'type': 'number', 'description': 'If provided, returns only the first N lines of the file'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=None),
 Tool(name='read_text_file', description="Read the complete contents of a file from the file system as text. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Use the 'head' parameter to read only the first N lines of a file, or the 'tail' parameter to read only the last N lines of a file. Operates on the file as text regardless of extension. Only work

# bring on the Agent with Tools!

In [9]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=files_params, client_session_timeout_seconds=600) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=600) as mcp_server_browser:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4.1-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)


I found a great recipe for Banoffee Pie and summarized it in markdown format. The file is named banoffee.md and contains the ingredients and method for making the pie, including preparation and cooking times. If you want, I can show you the content of the recipe summary.


### Check out the trace

https://platform.openai.com/traces

## MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



